# Lab 06 Softmax Zoo_classifier-eager

> - Softmax를 사용하여 Zoo 데이터를 활용하여 분류를 진행

### 기본 Library 선언 및 Tensorflow 버전 확인

In [1]:
import tensorflow as tf
import numpy as np

print(tf.__version__)

tf.random.set_seed(777)

2.3.0


In [16]:
xy = np.loadtxt('data/data-04-zoo.csv', delimiter=',', dtype=np.float32)
x_data = xy[:, 0:-1]
y_data = xy[:, -1]

nb_classes = 7

Y_one_hot = tf.one_hot(y_data.astype(np.int32), nb_classes)

print(x_data.shape, Y_one_hot.shape)

(101, 16) (101, 7)


In [21]:
W = tf.Variable(tf.random.normal((16, nb_classes)), name = 'weight')
b = tf.Variable(tf.random.normal((nb_classes, )), name = 'bias')
variables = [W, b]

# tf.nn.softmax computes softmax activations
# softmax = exp(logits) / reduce_sum(exp(logits), dim)
def logit_func(X):
    return tf.matmul(X, W) + b

def hypothesis(X):
    return tf.nn.softmax(logit_func(X))

def cost_func(X, Y):
    logits = logit_func(X)
    cost_i = tf.keras.losses.categorical_crossentropy(y_true = Y,
                                                      y_pred = logits,
                                                      from_logits=True)
    cost = tf.reduce_mean(cost_i)
    return cost

def grad_func(X, Y):
    with tf.GradientTape() as tape:
        loss = cost_func(X, Y)
        grads = tape.gradient(loss, variables)
        return grads
    
def prediction(X, Y):
    pred = tf.argmax(hypothesis(X), 1)
    correct_prediction = tf.equal(pred, tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    
    return accuracy

In [22]:
def fit(X, Y, epochs = 2000, verbose = 100):
    optimizer = tf.optimizers.SGD(learning_rate=0.1)
    
    for i in range(epochs):
        grads = grad_func(X, Y)
        optimizer.apply_gradients(zip(grads, variables))
        if (i==0) | ((i+1)%verbose==0):
            acc = prediction(X, Y).numpy()
            loss = cost_func(X, Y).numpy() 
            print('Steps: {} Loss: {}, Acc: {}'.format(i+1, loss, acc))
            
fit(x_data, Y_one_hot)

Steps: 1 Loss: 7.933250427246094, Acc: 0.059405941516160965
Steps: 100 Loss: 0.4977976381778717, Acc: 0.8613861203193665
Steps: 200 Loss: 0.31551873683929443, Acc: 0.9009901285171509
Steps: 300 Loss: 0.24008572101593018, Acc: 0.9504950642585754
Steps: 400 Loss: 0.19655703008174896, Acc: 0.9702970385551453
Steps: 500 Loss: 0.16733193397521973, Acc: 0.9702970385551453
Steps: 600 Loss: 0.1459740698337555, Acc: 0.9702970385551453
Steps: 700 Loss: 0.12952466309070587, Acc: 0.9900990128517151
Steps: 800 Loss: 0.11640135943889618, Acc: 0.9900990128517151
Steps: 900 Loss: 0.10566327720880508, Acc: 0.9900990128517151
Steps: 1000 Loss: 0.09670622646808624, Acc: 0.9900990128517151
Steps: 1100 Loss: 0.08912003040313721, Acc: 1.0
Steps: 1200 Loss: 0.08261377364397049, Acc: 1.0
Steps: 1300 Loss: 0.07697471976280212, Acc: 1.0
Steps: 1400 Loss: 0.07204264402389526, Acc: 1.0
Steps: 1500 Loss: 0.06769464164972305, Acc: 1.0
Steps: 1600 Loss: 0.06383457034826279, Acc: 1.0
Steps: 1700 Loss: 0.0603862181305